# Customer Churn Prediction

## i. Load Libraries

In [16]:
# General modules
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime, timedelta

# Scikit-learn modules
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

## 1. Database Setup and Data Loading

In [ ]:
# Database connection setup

DATABASE_FILE = "classicmodels.sqlite"
engine = create_engine(f"sqlite:///../data/{DATABASE_FILE}")

def get_data(sql_query):
    """Executes an SQL query and returns the results as a pandas DataFrame."""
    with engine.connect() as connection:
        try:
            df = pd.read_sql(text(sql_query), connection)
            return df
        except Exception as e:
            print(f"Database Query Error: {e}")
            return pd.DataFrame()

## 2. Feature Engineering (RFM and Churn Target)

In [ ]:
# Define the observation date (usually the day the analysis is run)
# We'll set a reference date a bit after the latest order in the database for a realistic look.
REFERENCE_DATE = datetime(2006, 1, 1) # Assuming data ends around 2005

# Define the Churn threshold (12 months of inactivity)
CHURN_THRESHOLD_DAYS = 365

In [ ]:
# 2.1. SQL Query to get base customer data and RFM components

# Recency: Days since last order
# Frequency: Total number of orders
# Monetary: Total sales amount
rfm_query = f"""
WITH CustomerRecency AS (
    SELECT
        customerNumber,
        MAX(orderDate) AS LastOrderDate,
        COUNT(orderNumber) AS Frequency
    FROM
        orders
    GROUP BY
        customerNumber
),
CustomerMonetary AS (
    SELECT
        o.customerNumber,
        SUM(od.quantityOrdered * od.priceEach) AS Monetary
    FROM
        orders o
    JOIN
        orderdetails od ON o.orderNumber = od.orderNumber
    GROUP BY
        o.customerNumber
)
SELECT
    c.customerNumber,
    c.country,
    c.creditLimit,
    strftime('%Y-%m-%d', LastOrderDate) AS LastOrderDateStr,
    t1.Frequency,
    t2.Monetary
FROM
    customers c
JOIN
    CustomerRecency t1 ON c.customerNumber = t1.customerNumber
JOIN
    CustomerMonetary t2 ON c.customerNumber = t2.customerNumber;
"""  # noqa: F541

df_raw = get_data(rfm_query)

In [29]:
# 2.2. Calculate Recency and Churn Target in Python
df_raw['LastOrderDate'] = pd.to_datetime(df_raw['LastOrderDateStr'])
df_raw['Recency'] = (REFERENCE_DATE - df_raw['LastOrderDate']).dt.days

# Create the binary target variable 'Churned'
# Churned = 1 if Recency > CHURN_THRESHOLD_DAYS, otherwise 0
df_raw['Churned'] = (df_raw['Recency'] > CHURN_THRESHOLD_DAYS).astype(int)

# Select features for the model
features = ['Recency', 'Frequency', 'Monetary', 'creditLimit', 'country']
target = 'Churned'
df_model = df_raw[features + [target]].copy()

print("--- Data Preparation Complete ---")
print(f"Total Customers: {len(df_model)}")
print(f"Churned Customers (Recency > {CHURN_THRESHOLD_DAYS} days): {df_model['Churned'].sum()}")
print("\n --- Legend ---")
print("Recency: Days since last order")
print("Frequency: Total number of orders")
print("Monetary: Total sales amount")
print("\n ---Sample Data: ---")
print(df_model.head())

--- Data Preparation Complete ---
Total Customers: 98
Churned Customers (Recency > 365 days): 54

 --- Legend ---
Recency: Days since last order
Frequency: Total number of orders
Monetary: Total sales amount

 ---Sample Data: ---
   Recency  Frequency   Monetary  creditLimit    country  Churned
0      402          3   22314.36      21000.0     France        1
1      398          3   80180.98      71800.0        USA        1
2      398          5  180585.07     117300.0  Australia        1
3      215          4  158573.12     118200.0     France        0
4      422          4  104224.79      81700.0     Norway        1


## 3. Preprocessing and Model Training

In [ ]:
# 3.1. Define Feature Types
numerical_features = ['Recency', 'Frequency', 'Monetary', 'creditLimit']
categorical_features = ['country']

In [17]:
# 3.2. Define Preprocessing Steps (ColumnTransformer)
# Standard Scaler for numerical data
# One-Hot Encoder for categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

In [18]:
# 3.3. Define the Model Pipeline
# The pipeline chains the preprocessing steps with the classifier
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

In [19]:
# 3.4. Split Data
X = df_model.drop(target, axis=1)
y = df_model[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [20]:
# 3.5. Train the Model
print("\n--- Training RandomForest Classifier ---")
model_pipeline.fit(X_train, y_train)
print("Training Complete.")


--- Training RandomForest Classifier ---
Training Complete.


## 4. Evaluation

In [21]:
# 4.1. Make Predictions
y_pred = model_pipeline.predict(X_test)

In [22]:
# 4.2. Evaluate Performance
print("\n--- Model Evaluation ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


--- Model Evaluation ---
Accuracy: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        17

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [23]:
# 4.3. Feature Importance (Extract from the trained pipeline)
# Get the names of the encoded features
ohe_feature_names = list(model_pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features))
all_feature_names = numerical_features + ohe_feature_names

In [24]:
# Get importance scores
importances = model_pipeline.named_steps['classifier'].feature_importances_
feature_importance_df = pd.DataFrame({'Feature': all_feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).head(10)

print("\nTop 10 Feature Importances:")
print(feature_importance_df)


Top 10 Feature Importances:
              Feature  Importance
0             Recency    0.540800
2            Monetary    0.132626
1           Frequency    0.126295
3         creditLimit    0.091538
20      country_Spain    0.013092
23        country_USA    0.012029
9     country_Finland    0.011679
4   country_Australia    0.010819
10     country_France    0.010727
11    country_Germany    0.007491


## 5. Model Use Cases: Churn Prediction Operations

### 5.1 Sample list of customers for churn prediction

In [31]:
# Simulate a small dataset of active customers with a generated Churn_Score
data = {
    'customerNumber': [103, 114, 119, 121, 141, 151, 175, 186, 201, 233],
    'customerName': ['Atelier graphique', 'Australian Collectors Co', 'La Rochelle Gifts', 'Baane Mini Imports', 'Euro+ Postales', 'Motor City Art', 'Gift Ideas Corp', 'Mini Wheels Co.', 'Online Diecast Creations', "L'Avenue des Cames"],
    'country': ['France', 'Australia', 'France', 'Norway', 'Spain', 'USA', 'USA', 'USA', 'USA', 'France'],
    'creditLimit': [71000.0, 117300.0, 80500.0, 81700.0, 118200.0, 103800.0, 89300.0, 75600.0, 66500.0, 95800.0],
    'Monetary': [22314.36, 180585.07, 72555.20, 104224.79, 158573.12, 105000.00, 45000.00, 15000.00, 90000.00, 55000.00],
    # SIMULATED CHURN SCORE (0 to 1)
    'Churn_Score': [0.15, 0.92, 0.45, 0.78, 0.55, 0.22, 0.88, 0.10, 0.41, 0.35]
}

df_active_customers = pd.DataFrame(data)
df_active_customers['customerNumber'] = df_active_customers['customerNumber'].astype(int)

print("--- Simulated Active Customer Data ---")
print(df_active_customers[['customerName', 'creditLimit', 'Monetary', 'Churn_Score']].head())
print("-" * 50)

--- Simulated Active Customer Data ---
               customerName  creditLimit   Monetary  Churn_Score
0         Atelier graphique      71000.0   22314.36         0.15
1  Australian Collectors Co     117300.0  180585.07         0.92
2         La Rochelle Gifts      80500.0   72555.20         0.45
3        Baane Mini Imports      81700.0  104224.79         0.78
4            Euro+ Postales     118200.0  158573.12         0.55
--------------------------------------------------


### 5.2. Sales Team: Targeting Outreach

In [32]:
# Define main function
def get_sales_action(score):
    """Determines the sales team's action based on the Churn Score."""
    if score >= 0.70:
        return "Immediate Sales Manager Contact (High-Risk Intervention)"
    elif score >= 0.40:
        return "Personalized Email Sequence (Mid-Risk Nurturing)"
    else:
        return "Standard Communication (Low-Risk Monitoring)"

# Apply the function to create a new column with the recommended action
df_active_customers['Sales_Action'] = df_active_customers['Churn_Score'].apply(get_sales_action)

print("\n--- Sales Team Prioritization ---")
df_sales_focus = df_active_customers[
    ['customerName', 'Churn_Score', 'Monetary', 'Sales_Action']
].sort_values(by='Churn_Score', ascending=False)
print(df_sales_focus)
print("-" * 50)


--- Sales Team Prioritization ---
               customerName  Churn_Score   Monetary  \
1  Australian Collectors Co         0.92  180585.07   
6           Gift Ideas Corp         0.88   45000.00   
3        Baane Mini Imports         0.78  104224.79   
4            Euro+ Postales         0.55  158573.12   
2         La Rochelle Gifts         0.45   72555.20   
8  Online Diecast Creations         0.41   90000.00   
9        L'Avenue des Cames         0.35   55000.00   
5            Motor City Art         0.22  105000.00   
0         Atelier graphique         0.15   22314.36   
7           Mini Wheels Co.         0.10   15000.00   

                                        Sales_Action  
1  Immediate Sales Manager Contact (High-Risk Int...  
6  Immediate Sales Manager Contact (High-Risk Int...  
3  Immediate Sales Manager Contact (High-Risk Int...  
4   Personalized Email Sequence (Mid-Risk Nurturing)  
2   Personalized Email Sequence (Mid-Risk Nurturing)  
8   Personalized Email Sequen

In [33]:
# --- 5.3. Marketing Team: Campaign Budget Allocation ---

# We'll isolate the High-Risk customers for a focused retargeting campaign.
HIGH_RISK_THRESHOLD = 0.70

df_high_risk_marketing = df_active_customers[
    df_active_customers['Churn_Score'] >= HIGH_RISK_THRESHOLD
].sort_values(by='Churn_Score', ascending=False)

print("\n--- Marketing Retargeting Campaign List (Score >= 0.70) ---")
print(df_high_risk_marketing[['customerName', 'Churn_Score', 'country']])
print("-" * 50)


--- Marketing Retargeting Campaign List (Score >= 0.70) ---
               customerName  Churn_Score    country
1  Australian Collectors Co         0.92  Australia
6           Gift Ideas Corp         0.88        USA
3        Baane Mini Imports         0.78     Norway
--------------------------------------------------


In [34]:
# --- 5.4. Finance Team: Credit Limit Review (Proactive Review) ---

# Rule: If a customer is high-risk AND has a high credit limit, recommend a review/delay increase.
FINANCE_RISK_THRESHOLD = 0.75
HIGH_LIMIT_THRESHOLD = 90000.00 # Define 'high' credit limit based on overall distribution

def get_finance_review(row):
    """Determines if the finance team needs to review the account/credit limit."""
    if row['Churn_Score'] >= FINANCE_RISK_THRESHOLD and row['creditLimit'] >= HIGH_LIMIT_THRESHOLD:
        return f"URGENT REVIEW: HIGH Risk & HIGH Limit. Reduce Exposure."
    elif row['Churn_Score'] >= FINANCE_RISK_THRESHOLD:
        return f"Review Required: HIGH Risk."
    else:
        return "Standard Status."

In [35]:
# Apply the function
df_active_customers['Finance_Review_Status'] = df_active_customers.apply(get_finance_review, axis=1)

print("\n--- Finance Team Credit Review List ---")
df_finance_review = df_active_customers[
    df_active_customers['Finance_Review_Status'] != 'Standard Status.'
].sort_values(by='Churn_Score', ascending=False)

print(df_finance_review[['customerName', 'Churn_Score', 'creditLimit', 'Finance_Review_Status']])
print("-" * 50)


--- Finance Team Credit Review List ---
               customerName  Churn_Score  creditLimit  \
1  Australian Collectors Co         0.92     117300.0   
6           Gift Ideas Corp         0.88      89300.0   
3        Baane Mini Imports         0.78      81700.0   

                               Finance_Review_Status  
1  URGENT REVIEW: HIGH Risk & HIGH Limit. Reduce ...  
6                        Review Required: HIGH Risk.  
3                        Review Required: HIGH Risk.  
--------------------------------------------------
